In [5]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from urllib.parse import unquote
from bs4 import BeautifulSoup
import random
from tqdm.notebook import tqdm
import re
import time

In [1]:
import pandas as pd 

In [6]:
def open_linkedin():
    #в этом файле мои логин и пароль
    with open('linkedin_login.txt', 'r') as handler:
        lines = handler.readlines()
        MAIL, PASS = lines[0].strip(), lines[1]

    driver = uc.Chrome()
    caps = DesiredCapabilities().CHROME
    caps['pageLoadStrategy'] = 'eager'
    #driver = webdriver.Chrome()
    # Время для включения ВПН
    time.sleep(20)
    
    driver.get("https://linkedin.com/uas/login")
    time.sleep(3)
    login_field = driver.find_element(By.ID,"username")
    login_field.send_keys(MAIL)

    pass_field = driver.find_element(By.ID,"password")
    pass_field.send_keys(PASS)
    pass_field.submit()
    time.sleep(30)  # 4 -> 60
    
    return driver

## Функция получения сотрудников компаний

In [7]:
def get_company_emploeeys(driver, companies:list, n_scrolls=1):
    #
    companies_employeers = {}
    for company in companies: 
        employees_links = []
        driver.get(f'https://www.linkedin.com/company/{company}/people/') 
        
        time.sleep(15) 
        # scroll + timeout 
        for _ in range(n_scrolls):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(random.uniform(5,10)) 
        soup = BeautifulSoup(driver.page_source, 'lxml')
        # org-people-profile-card__profile-info
        
        #сбор ссылок
        for index, person in enumerate(soup.find_all('div', attrs={'class': 'org-people-profile-card__profile-info'})):
            #print(index)
            try:
                image = person.find('div', attrs={'class':'artdeco-entity-lockup__image'})
                link = image.find('a', attrs={'class': 'app-aware-link'})['href']
                employees_links.append(link)
            except:
                continue
        companies_employeers[company] = employees_links
        time.sleep(random.uniform(1,5))  
          
    return companies_employeers
    

In [8]:
#Список компаний
company_list = ['sberbank', 'llc-internet-solutions-ozon-ru-', 'alfa-bank']

In [36]:
%%time

#открываем сайт
driver = open_linkedin()
#ссылки на страницы сотрудников
links = get_company_emploeeys(driver, companies=company_list, n_scrolls=2)

print(links)

{'sberbank': ['https://www.linkedin.com/in/vlad-semenov-7765751a7?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADBQyQoBG8VsUm0KhxiYFoHVgiO_B2miCKg', 'https://www.linkedin.com/in/pavel-kalita?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADBmXdEBIsvSgUFPhn4hYyR_eSUAsdb-sXc', 'https://www.linkedin.com/in/sergey-b-b6a604198?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC5dKYIBvM-GziCa-9a6umZEY5iO8Z_LXvU', 'https://www.linkedin.com/in/shagidullink?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAe5HmgBk5vYd8vI_PVeDNQLvoxoO2I5-WA', 'https://www.linkedin.com/in/germangolubev?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAODmvgBP71XINuowqr54O7y0lOGypsMnF0', 'https://www.linkedin.com/in/gtrondin?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAClixMQB6f781Hd8l_WldVVqGP7XDh8w2o8', 'https://www.linkedin.com/in/gleb-pilipenka-02b87a160?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACaSWqMBvCoB4lvKCQ9-POGxE2tExMqMYZk', 'https://www.linkedin.com/in/mikhail-morgunov-b07835108?miniProfileUrn=u

Полученные ссылки на профили по компаниям.

In [59]:
links_df = pd.DataFrame(links[company_list[0]],columns = [company_list[0]]) 
links_df[company_list[1]] = pd.Series(links[company_list[1]])
links_df[company_list[2]] = pd.Series(links[company_list[2]])

links_df

,sberbank,llc-internet-solutions-ozon-ru-,alfa-bank
0,https://www.linkedin.com/in/vlad-semenov-77657...,https://www.linkedin.com/in/maksim-savenkov-89...,https://www.linkedin.com/in/artur-nurutdinov-1...
1,https://www.linkedin.com/in/pavel-kalita?miniP...,https://www.linkedin.com/in/dmitriy-bolshakov-...,https://www.linkedin.com/in/olga-gebremariam-3...
2,https://www.linkedin.com/in/sergey-b-b6a604198...,https://www.linkedin.com/in/filipp-sergeev-b59...,https://www.linkedin.com/in/ruslan-nizamutdino...
3,https://www.linkedin.com/in/shagidullink?miniP...,https://www.linkedin.com/in/inga-baranets-0581...,https://www.linkedin.com/in/%F0%9F%9A%80-igor-...
4,https://www.linkedin.com/in/germangolubev?mini...,https://www.linkedin.com/in/yuliya-bulginova-7...,https://www.linkedin.com/in/anna-nikolaeva-188...
5,https://www.linkedin.com/in/gtrondin?miniProfi...,https://www.linkedin.com/in/petr-dytko-b3a7072...,https://www.linkedin.com/in/aleksey-latakhin?m...
6,https://www.linkedin.com/in/gleb-pilipenka-02b...,https://www.linkedin.com/in/asvezhentsev?miniP...,https://www.linkedin.com/in/alexander-permyako...
7,https://www.linkedin.com/in/mikhail-morgunov-b...,https://www.linkedin.com/in/stepan-kulagin-339...,https://www.linkedin.com/in/aleksey-solovyev?m...
8,https://www.linkedin.com/in/anastasia-beltyuko...,https://www.linkedin.com/in/mikhail-sokolov-3a...,https://www.linkedin.com/in/arsen-kurmashev-5a...
9,https://www.linkedin.com/in/%D0%B2%D0%BB%D0%B0...,https://www.linkedin.com/in/dmitrytyugaev?mini...,https://www.linkedin.com/in/andnkomarov?miniPr...


In [73]:
driver = open_linkedin()
scrape_posts_reactions(driver, 'https://www.linkedin.com/in/nurlan-isbulaev-34bb211a3/recent-activity/all/')

9


['🔥 11 сервисов на базе ИИ ,которые помогут прокачать твой профиль в LinkedIn🔥❗ Лучшая благодарность за мой труд - это ваши реакции и комментарии!🔥Буду благодарен за подписку на мои другие социальные сети:✔Instagram: instagram.com/ralkofood\xa0Там все о еде, обратная сторона работы ресторанов, рецепты, Food-обзоры, будет много контента о жизни в Польше и Европе в целоми т.д.✔Telegram: t.me/RalkoDenis\xa0Там дублирую весь контент из LinkedIn, что бы вам было удобней читать и ничего не пропускать.📌\xa0jasper.ai/\xa0ИИ-копирайтер. Поможет сгенерировать интересные статьи в LinkedIn\xa0#linkedin📌\xa0wonsulting.com/networkai\xa0Автоматизированный инструмент ИИ, который помогает пользователям быстро и эффективно расширить свою профессиональную сеть.📌\xa0sayhi.pro/\xa0Расширение Chrome на базе ИИ, которое помогает пользователям писать эффективные и убедительные сообщения LinkedIn, адаптированные к их получателям.📌\xa0linkoai.com/\xa0Инструмент на основе ИИ, который помогает людям максимально и

In [63]:
def profiles_search_results(driver, query:str, pages_num=5):
    #сюда складываю ссылки на профили
    profiles_links = []
    for page in range(1, pages_num+1):
        # нажатие кнопки next  у меня не работает, поэтому просто меняю страницу в самой url
        # driver.get(f'https://www.linkedin.com/search/results/people/?keywords={query}'+
        #            f'&origin=SWITCH_SEARCH_VERTICAL&page={page}&sid=8TE')
        # локация россия
        driver.get(f'https://www.linkedin.com/search/results/people/?geoUrn=%5B%22101728296%22%5D&keywords={query}' +
                   f'&origin=FACETED_SEARCH&page={page}&sid=wMe')
        soup = BeautifulSoup(driver.page_source, 'lxml')
        # тут просто собираю ссылки
        for person in soup.find_all('span', attrs={'class': 'entity-result__title-text'}):
            link = person.find('a', attrs={'class': 'app-aware-link'})['href']
            profiles_links.append(link)
        # прокрутка до конца страницы для имитации реального пользователя
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(1,5))
    
    #driver.close()
        
    return profiles_links

In [64]:
def get_company_and_experience(driver)-> float:
    #переходим на страницу опыта
    experience_page = driver.current_url + 'details/experience/'
    driver.get(experience_page)
    time.sleep(random.uniform(1,5))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    soup = BeautifulSoup(driver.page_source,'lxml')
    total = 0
    #вот тут будут строчки из каждой позиции в опыте
    for i in soup.find_all('span', {'class':'t-14 t-normal t-black--light'}):
        # собираю тексты
        t = i.find('span', {'class': 'visually-hidden'}).text
        #не всегда есть год, месяц, иногда вообще пусто
        try:
            total += int(re.search(r'\d+ mo', t)[0].split()[0])
        except:
            pass
        try:
            total += int(re.search(r'\d+ yr', t)[0].split()[0]) * 12
        except:
            pass
    try:
        curr_job = (soup.find_all('li', {'class':'pvs-list__paged-list-item'})[0]
                         .find('span', 't-14 t-normal')
                         .find('span', 'visually-hidden')).text.strip()
        curr_company = curr_job.split(' · ')[0]
    except: 
        curr_job = None
        curr_company = None
    try:
        employment_type = curr_job.split(' · ')[1]
    except:
        employment_type = 'unknown'
        
    years = round(total / 12, 2)
    driver.back()
    return curr_company, years, employment_type

In [65]:
def get_skills(driver):
    curr_link = driver.current_url
    
    driver.get(curr_link + 'details/skills/')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source,'lxml')
    
    skills = []
    raw_skills = soup.find('main', 'scaffold-layout__main').find_all('div', 'pvs-entity')
    for skill in raw_skills:
        skills.append(skill.find('span', 'visually-hidden').text)
    driver.back()    
    return list(set(skills))

In [66]:
def get_profile_info(driver, link):
    driver.get(link)
    time.sleep(random.uniform(2,5))
    soup = BeautifulSoup(driver.page_source,'lxml')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    person = {}
    u_id = unquote(link.split('in/')[1].split('?')[0])
    person['name'] = soup.find('h1', {'class': 'text-heading-xlarge'}).text
    person['city'] = soup.find('span',"text-body-small inline t-black--light break-words").text.strip().split(',')[0]
    try:
        person['connections'] = re.search('\d+. connections|\d+ connections', 
                                          soup.find('div', 'ph5')
                                .text)[0].split()[0]
    except:
        person['connections'] = '0'
    role = soup.find('div', {'class': 'text-body-medium break-words'}).text.strip()
    person['role'] = (role.split(' at ')[0]
                      .split(' - ')[0]
                      .split(' – ')[0]
                      .split(' as ')[0])
    
    person_info = get_company_and_experience(driver)
    person['company'] = person_info[0]
    person['experience'] = person_info[1]
    person['employment_type'] = person_info[2]
    
    person['skills'] = get_skills(driver)

    return u_id, person

In [72]:
def scrape_posts_reactions(driver, link, num_scrolls=5):
    driver.get(link)
    time.sleep(random.uniform(3,5))
    last_height = driver.execute_script("return document.body.scrollHeight")
    #прокрутка до конца страницы
    for i in range(num_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(1,5))
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    # собираю тексты
    soup = BeautifulSoup(driver.page_source,'lxml')
    posts = soup.find_all('li', class_='profile-creator-shared-feed-update__container')
    posts_texts = []
    for post in posts:
        try:
            post_text = (post.find('div', {'class': 'feed-shared-update-v2__description-wrapper mr2'})
                         .find('span', {'dir': 'ltr'}).text)
            posts_texts.append(post_text)
        except:
            pass

    return posts_texts 

In [68]:
def get_posts_reactions(driver, link):
    #тут есть два раздела: posts и reactions
    time.sleep(random.uniform(1,5))
    
    posts_url = driver.current_url + 'recent-activity/all/'
    reactions_url = driver.current_url + 'recent-activity/reactions/'
    # тексты для каждого
    posts = scrape_posts_reactions(driver, posts_url)
    time.sleep(random.uniform(1,5))
    react_posts = scrape_posts_reactions(driver, reactions_url)
        
    return posts, react_posts

In [27]:
#test
NUM_PAGES  = 1
BREAK_POINT = 10
#
# NUM_PAGES  = 100
# BREAK_POINT = 1000

In [ ]:
%%time

#открываем сайт
driver = open_linkedin()
#ссылки на страницы сотрудников
links = get_company_emploeeys(driver, companies=company_list, n_scrolls=2)

print(links)

{'sberbank': ['https://www.linkedin.com/in/vlad-semenov-7765751a7?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADBQyQoBG8VsUm0KhxiYFoHVgiO_B2miCKg', 'https://www.linkedin.com/in/pavel-kalita?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADBmXdEBIsvSgUFPhn4hYyR_eSUAsdb-sXc', 'https://www.linkedin.com/in/sergey-b-b6a604198?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC5dKYIBvM-GziCa-9a6umZEY5iO8Z_LXvU', 'https://www.linkedin.com/in/shagidullink?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAe5HmgBk5vYd8vI_PVeDNQLvoxoO2I5-WA', 'https://www.linkedin.com/in/germangolubev?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAODmvgBP71XINuowqr54O7y0lOGypsMnF0', 'https://www.linkedin.com/in/gtrondin?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAClixMQB6f781Hd8l_WldVVqGP7XDh8w2o8', 'https://www.linkedin.com/in/gleb-pilipenka-02b87a160?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACaSWqMBvCoB4lvKCQ9-POGxE2tExMqMYZk', 'https://www.linkedin.com/in/mikhail-morgunov-b07835108?miniProfileUrn=u

In [ ]:
for company in links.keys():
    print(company)
    for count, link in tqdm(enumerate(links[company])):
        u_id, profile = get_profile_info(driver, link)
        profiles[u_id] = profile
        posts, reactions = get_posts_reactions(driver, link)
        u_posts[u_id] = posts
        u_reactions[u_id] = reactions
        if count==BREAK_POINT:  #5
            break
        
    len(profiles)

In [29]:
driver.close()